# The London Railway Network

In [1]:
from abc import ABC, abstractmethod  

class AbstractLondonRailwayMapper(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # data initialisation
    @abstractmethod
    def loadStationsAndLines(self):
        pass

    """
    returns the minimum number of stops to connect station "fromS" to station  "toS"
    fromS : str
    toS : str
    numStops : int
    """
    @abstractmethod
    def minStops(self, fromS, toS):     
        numStops = -1
        return numStops    
    
    """
    returns the minimum distance in miles to connect station "fromS" to station  "toS"
    fromS : str
    toS : str
    minDistance : float
    """
    @abstractmethod
    def minDistance(self, fromS, toS):
        minDistance = -1.0
        return minDistance
    
    """
    given an unordered list of station names, returns a new railway line 
    (represented as a list of adjacent station names), connecting all such stations 
    and such that the sum of the distances (in miles) between adjacent stations is minimised
    inputList : set<str>
    outputList : list<str>
    """
    @abstractmethod
    def newRailwayLine(self, inputList):
        outputList = []
        return outputList

In [2]:
# DATA STRUCTURE DEFINITIONS AND HELPER CODE

import heapq
from math import *

INFINITY = inf # math.inf
stationsFile = "londonstations.csv"
railwaysFile = "londonrailwaylines.csv"


"""
The London railway network is represented as a weighted undirected graph in my program. Where each vertex represents a station, an edge between two vertices represents a direct connection between those stations and the weight represents the distance in miles between the two stations. I have implemented this weighted undirected graph as a pair of nested dictionaries where the key of the outer dictionary is a vertex (A) and its value is a dictionary. The inner dictionaries keys are all the vertices adjacent to A and the value is the weight of the edges. For example, if I wanted the weight of edge AB, I would write ‘graph[A][B]’.
"""
class WeightedGraph:
    def __init__(self):
        self.graph = {}

    def addVertex(self, vertex):
        self.graph[vertex] = {}

    def addEdge(self, a, b, weight):
        if a in self.graph and b in self.graph:
            self.graph[a][b] = weight
            self.graph[b][a] = weight

    def getAdjacencyOfVertex(self, vertex):
        if vertex in self.graph:
            return self.graph[vertex]

    def getWeightOfEdge(self, a, b):
        if a in self.graph and b in self.graph:
            return self.graph[a][b]



"""
Along with storing the railway network as a graph we also must store some data on each station. My program does this by making use of a station class, which stores a stations name, its longitude/latitude points, and the lines it’s on (stored as an array). An instance of the station class is created for every station and this instance is stored in a dictionary called ‘allStations’ (below) with the station name being the key. For example ‘allStations[“Oxford Station”].longitude’ fetches the longitude of the station Oxford Circus.
"""
class Station:
    def __init__(self, name, latitude, longitude):
        self.name = name
        self.lines = []
        self.latitude = latitude
        self.longitude = longitude

    def addLineToLines(self, line):
        if line not in self.lines:
            self.lines.append(line)



"""
The Queue class which is used in the 'minStops()' BFS implementation.
"""
class Queue:
    def __init__(self):
        self.arr = []
    
    def enqueue(self, item):
        self.arr.append(item)
        self.head = 0

    def dequeue(self):
        item = self.arr[self.head]
        self.head +=1
        if self.head>15:
            self.arr = self.arr[15:]
            self.head = 0
        return item

    def isEmpty(self):
        return len(self.arr)==0



"""
Dijkstra’s algorithm makes use of a priority queue and there are many ways to implement a priority queue. Some implementations have a better theoretical time complexity than others but only in certain scenarios. To get a better understanding of how these different implementations would affect my runtime I coded a priority queue using a list, a dictionary, a binary heap and using the heapq module. I ran the Dijkstra’s algorithm each of these implementations my generating random stations and then plotted these results on a graph. From these results I decided to go with a heapq implementation as it had the quickest execution time. 
"""
class PriorityQueue:
    def __init__(self):
        self.arr = []
    
    def enqueue(self, item, priority):
        heapq.heappush(self.arr, (priority, item))

    def dequeue(self):
        value = heapq.heappop(self.arr)
        return value[1] 

    def contains(self, item):
        for i in range(len(self.arr)):
            if self.arr[i][1]==item:
                return i
        return -1

    def decreaseKey(self, item, priority, index):
        self.arr[index] = (priority, item)
        heapq.heapify(self.arr)

    def isEmpty(self):
        return len(self.arr)==0


In [3]:
# MAIN API

import csv

class LondonRailwayMapper(AbstractLondonRailwayMapper):
    
    def __init__(self):

        # implements the railway network as an undirected weighted graph
        self.networkGraph = WeightedGraph()

        # stores all the information about each station
        self.allStations = {}           
     


    """ 
    When the ‘loadStationsAndLines()’ function is called it first loops through all the 
    rows in ‘londonstations.csv’. It then creates a station object for each station along 
    with adding the station as a vertex to the network graph. 
    """    
    def loadStationsAndLines(self):

        # load all the stations first
        with open(stationsFile) as csvfile:
            stations = list(csv.reader(csvfile, delimiter=','))
            for row in stations[1:]:
                stationName = row[0]

                # stations object is created and stored
                station = Station(stationName, float(row[1]), float(row[2])) 
                self.allStations[stationName] = station

                # the station is added to the network graph
                self.networkGraph.addVertex(stationName)
        
        # make connections between the stations
        with open(railwaysFile) as csvfile:
            railwayLines = list(csv.reader(csvfile, delimiter=','))
            for row in railwayLines[1:]:
                line = row[0]
                station1 = row[1]
                station2 = row[2]

                # add the line which a station is on the the station object which holds all the 
                # information about a station
                self.allStations[station1].addLineToLines(line)
                self.allStations[station2].addLineToLines(line)

                # use the haversine formula to calculate the distance between two stations 
                # add the conncection between the two station to the network graph as an edge 
                # with the distance between them being the weight of the edge
                distance = self.distanceBetween(station1, station2)
                self.networkGraph.addEdge(station1, station2, distance)


    
    """ 
    Haversine formula to work out the distance between two station 'a' and 'b'. 
    Used the source below to construct this function:
       https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
    """
    def distanceBetween(self, a, b): 
        lon1, lat1, lon2, lat2 = map(radians, self.getCoordinatesOf(a, b))
        dlon = lon2 - lon1 
        dlat = lat2 - lat1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = asin(sqrt(a)) 
        radiusOfEarth = 3950 # radius of Earth in miles
        return c * radiusOfEarth * 2


    """
    Returns the longitude and latitude of two stations as an array. Used in the distanceBetween() function above.
    """
    def getCoordinatesOf(self, a, b):
        lon1 = self.allStations[a].longitude
        lat1 = self.allStations[a].latitude
        lon2 = self.allStations[b].longitude
        lat2 = self.allStations[b].latitude
        return [lon1, lat1, lon2, lat2]


    """
    The ‘minStops()’ function is tasked with finding the minimum number of stops between 
    two stations. My program computes this using a modified version of the breadth-first 
    search algorithm (BFS.) The BFS algorithm traverses a graph by visiting all neighbouring 
    nodes in increasing distance from the start node.

    At the beginning, I create a distance to source (‘distToSource’) dictionary, where the 
    key is the station name, and the value is the distance to the source (initially it is 
    -1.) In a regular BFS algorithm there is usually a marked data structure which tracks 
    whether a vertex has been visited or not, however, in my implementation I don’t need this 
    as if the distance to source is -1 for any station this indirectly implies that the station 
    hasn’t been visited.

    An ‘edgeTo’ data structure is also usually included in a regular BFS but I can exclude it in 
    my implementation as in the specification for ‘minStops()’ it states that we only need to 
    return the number of minimum stops and not the actual path (however, I did use it when 
    debugging to ensure I was getting the correct answer.) 
    """
    def minStops(self, fromS, toS):     

        # innitialize data structures
        distToSource = {}
        startInNetwork = False
        endInNetwork = False
        for station in self.allStations:
            distToSource[station] = -1
            if station==fromS:
                startInNetwork=True
            if station==toS:
                endInNetwork = True
        
        # validation - ensure start and end stations are in the graph, by doing this I don't
        # have to call BFS on the whole graph unnecessarily which reduces the number of time 
        # the worst case scenario can happen
        if startInNetwork and endInNetwork:

            # breadth-first search algorithm

            distToSource[fromS] = 0
            queue = Queue()
            queue.enqueue(fromS)
            while not queue.isEmpty():
                currentStation = queue.dequeue()
                for station in self.networkGraph.getAdjacencyOfVertex(currentStation):

                    # if the station hasn't already been visited then add it to the queue
                    # and update its distance to source (distToSource) value to be 1 more
                    # then the station you just came from

                    if distToSource[station]==-1: 
                        queue.enqueue(station)
                        distToSource[station] = distToSource[currentStation]+1

                    # if the station you just visited is the end station then return the
                    # number of stops between the start and the end station

                    if station==toS:
                        return distToSource[toS]
        
        return -1   # if either stations aren't in the graph then return -1  
    

    
    """
    The ‘minDinstance()’ function is tasked with finding the minimum distance between 
    two stations. In a standard railway network, there are no negative weights, so my 
    program makes use of Dijkstra’s algorithm, with some modifications, to compute the 
    minimum distance.

    Similar to the ‘minStops()’ function, at the beginning, I create a distance to source 
    (‘distToSource’) dictionary, where the key is the station name, and the value is the 
    distance to the source (initially it is infinity.) And for the same reasons as above 
    I have excluded an ‘edgeTo’ data structure. 
    """
    def minDistance(self, fromS, toS):

        # innitialize data structures
        distToSource = {}
        startInGraph = False
        endInGraph = False
        for station in self.allStations:
            distToSource[station] = INFINITY
            if station==fromS:
                startInGraph=True
            if station==toS:
                endInGraph = True

        # validation - ensure start and end stations are in the graph
        if startInGraph and endInGraph:

            # Dijkstra's algorithm

            distToSource[fromS] = 0
            pq = PriorityQueue()
            pq.enqueue(fromS, 0)
            while not pq.isEmpty():
                currentStation = pq.dequeue()
                for station in self.networkGraph.getAdjacencyOfVertex(currentStation):
                    distance = self.networkGraph.getWeightOfEdge(station,currentStation)

                    # if there is a shorter way to get to a staion then update the distance
                    # to the new shorter distance i.e realax the edges
                    # also add/update the stations value in the priority queue

                    if distToSource[station] > (distToSource[currentStation] + distance):
                        distToSource[station] = distToSource[currentStation] + distance
                        index = pq.contains(station)
                        if index>=0:
                            pq.decreaseKey(station, distToSource[station], index)
                        else:
                            pq.enqueue(station, distToSource[station])

            return round(distToSource[toS], 4) # return the distance between the two stations rounded to 4 d.p.

        return -1 # if either stations aren't in the graph then return -1
    
    
    
    """
    The ‘minRailwayLine()’ function is tasked with finding a way to connect a list of 
    stations such that the distance of the path across them is as optimal as possible. 
    My function does this using a combination of different techniques but mainly relies of 
    the nearest neighbour algorithm and the 2-opt optimization procedure. Both are known 
    heuristics used to solve the traveling salesman problem, but I have made modifications 
    to adapt them to our specific problem.

    I make use a two slightly different algorithms to compute the new railway line. One algorithm 
    gives a more optimal solution, but it doesn’t scale very well while the other gives a slightly 
    less optimal solution but scales better. If the length of the input list is greater than 60 
    than the algorithm which scales better is used, else the algorithm which gives a better solution is 
    used. The crossover point is 60 stations as after much testing I realized that this number gives the 
    best solution for both the algorithms. 
    """
    def newRailwayLine(self, inputList):
        if len(inputList)<=60:
            return self.newRailwayLineA(inputList)
        else:
            return self.newRailwayLineB(inputList)



    def newRailwayLineA(self, stationSet):
        minLength = INFINITY
        minPath = []
        for i in range(len(stationSet)): # computes nearest neighbour using every possible start station
            length, path = self.nearestNeighbour(stationSet, stationSet[i], minLength)
            if length<minLength:
                minLength = length
                minPath = path
        return self.k2OptFurther(minPath)[1] # we just need the path so we can slice it off and return it



    def newRailwayLineB(self, stationSet):
        minLength = INFINITY
        minPath = []
        directions = self.northernAndSouthernMostStations(stationSet) + self.westernAndEasternMostStations(stationSet) 
        for i in range(4): # computes nearest neighbour only on the North, South, East and West most stations
            length, path = self.nearestNeighbour(stationSet, directions[i], minLength)
            if length<minLength:
                minLength = length
                minPath = path
        return self.k2Opt(minPath)[1] # we just need the path so we can slice it off and return it



    def lengthOfPath(self, path):
        total = 0
        for i in range(len(path)-1):
            total += self.distanceBetween(path[i], path[i+1])
        return total


    """
    The nearest neighbour algorithm used in the 'newRailwayLine()' function. 
    """
    def nearestNeighbour(self, inputSet, startNode, minSoFar):

        # copy the inputSet into a new variable so we can reuse it
        # as we will be deleting items from it
        stations = inputSet.copy()

        # innitialize the total and path
        total = 0
        path = []
        current = startNode
        path.append(current)
        stations.remove(current)

        # nearest neighbour algorithm
        for _ in range(len(stations)):

            minDistance = INFINITY
            nextStation = ""

            # finds the nearest station compared to the current station
            for station in stations:
                distance = self.distanceBetween(current, station)
                if distance<minDistance:
                    minDistance = distance
                    nextStation = station
            total += minDistance

            # if the total is already greater than the current minimum distance there is 
            # no need to keep going
            if total>minSoFar:
                return INFINITY, None

            # otherwise add the nearest neighbour to the path and continue
            current = nextStation
            path.append(current)
            stations.remove(current)

        return total, path # return the length of the path and the path


    
    """
    Find the North and South most stations in a list of stations. It does this by
    sorting all the stations by its latitude.
    """
    def northernAndSouthernMostStations(self, stationSet):
        latitudesOfStations = []
        for station in stationSet:
            stationLatitude = float(self.allStations[station].latitude)
            latitudesOfStations.append((stationLatitude, station))
        latitudesOfStations.sort()
        return [latitudesOfStations[len(stationSet)-1][1], latitudesOfStations[0][1]]

    
    """
    Find the West and East most stations in a list of stations. It does this by
    sorting all the stations by its longitude.
    """
    def westernAndEasternMostStations(self, stationSet):
        longitudesOfStations = []
        for station in stationSet:
            stationLongitude = float(self.allStations[station].longitude)
            longitudesOfStations.append((stationLongitude, station))
        longitudesOfStations.sort()
        return [longitudesOfStations[len(stationSet)-1][1], longitudesOfStations[0][1]]

    

    """
    The 2-opt Optimaization technique used in the 'newRailwayLine()' function. 
    This repeatedly swaps over two points on a path to find a shorter path.
    """
    def k2Opt(self, inputPath):

        # innitialize
        minDistance = self.lengthOfPath(inputPath)
        path = inputPath

        # 2-opt algortihm
        for i in range(1, len(inputPath)):
            for k in range(i+1, len(inputPath)):

                # swaps a section of a path over then check if this path was shorter
                # than the original path
                newPath = self.k2OptSwap(path, i, k)
                distance = self.lengthOfPath(newPath)

                # if this swapped path is shorter than it replaces it
                if distance<minDistance:
                    minDistance = distance
                    path = newPath

        return minDistance, path



    """
    Reverses a section on a path and returns this new path, used in the
    2-opt optimization algorithm.
    """
    def k2OptSwap(self, inputPath, i, k):
        newRoute = inputPath[:i]
        swapped = inputPath[i:k]
        swapped.reverse()
        return newRoute+swapped+inputPath[k:]


    """
    This is used in 'newRailwayLineA()' to optimize a path. Essentially, the 
    the 2-opt algorithm is called on the input path and if it becomes shorter
    than the 2-opt algorithm is called on it again to further optimize it.
    """
    def k2OptFurther(self, inputPath):
        length, path = self.k2Opt(inputPath)
        if inputPath!=path:
            length, path = self.k2Opt(path)
        return length, path

In [4]:
# TESTING

import timeit

testMapper = LondonRailwayMapper()

# testing the loadStationsAndLines() API 
starttime = timeit.default_timer()
testMapper.loadStationsAndLines()
endtime = timeit.default_timer()
print("\nExecution time to load:", round(endtime-starttime,3))

# testing the minStops() and minStops() API on a sample of from/to station pairs  
fromList = ["Baker Street", "Epping", "Canonbury", "Vauxhall"]
toList = ["North Wembley", "Belsize Park", "Balham", "Leytonstone"]

for i in range(len(fromList)):
    starttime = timeit.default_timer()
    stops = testMapper.minStops(fromList[i], toList[i])
    endtime = timeit.default_timer()
    print("\nExecution time minStops:", round(endtime-starttime,3))

    starttime = timeit.default_timer()
    dist = testMapper.minDistance(fromList[i], toList[i])
    endtime = timeit.default_timer()
    print("Execution time minDistance:", round(endtime-starttime,3))

    print("From", fromList[i], "to", toList[i], "in", stops, "stops and", dist, "miles")  
    
# testing the newRailwayLine() API on a small list of stations  
stationsList = ["Queens Park", "Chigwell", "Moorgate", "Swiss Cottage", "Liverpool Street", "Highgate"]

starttime = timeit.default_timer()
newLine = testMapper.newRailwayLine(stationsList)
endtime = timeit.default_timer()

print("\n\nStation list", stationsList)
print("New station line", newLine)
print("Total track length from", newLine[0], "to", newLine[len(newLine)-1], ":", testMapper.minDistance(newLine[0], newLine[len(newLine)-1]), "miles")
print("Execution time newLine:", round(endtime-starttime,3))

# testing the newRailwayLine() API on a big list of stations  
stationsList = ["Abbey Road", "Barbican", "Bethnal Green", "Cambridge Heath", "Covent Garden", "Dollis Hill", "East Finchley", "Finchley Road and Frognal", "Great Portland Street", "Hackney Wick", "Isleworth", "Kentish Town West", "Leyton", "Marble Arch", "North Wembley", "Old Street", "Pimlico", "Queens Park", "Richmond", "Shepherds Bush", "Tottenham Hale", "Uxbridge", "Vauxhall", "Wapping"]

starttime = timeit.default_timer()
newLine = testMapper.newRailwayLine(stationsList)
endtime = timeit.default_timer()

print("\n\nStation list", stationsList)
print("New station line", newLine)
print("Total track length from", newLine[0], "to", newLine[len(newLine)-1], ":", testMapper.minDistance(newLine[0], newLine[len(newLine)-1]), "miles")
print("Execution time newLine:", round(endtime-starttime,3))


Execution time to load: 0.016

Execution time minStops: 0.001
Execution time minDistance: 0.004
From Baker Street to North Wembley in 6 stops and 8.2095 miles

Execution time minStops: 0.002
Execution time minDistance: 0.004
From Epping to Belsize Park in 17 stops and 20.6039 miles

Execution time minStops: 0.004
Execution time minDistance: 0.005
From Canonbury to Balham in 10 stops and 8.8206 miles

Execution time minStops: 0.001
Execution time minDistance: 0.007
From Vauxhall to Leytonstone in 6 stops and 8.8249 miles


Station list ['Queens Park', 'Chigwell', 'Moorgate', 'Swiss Cottage', 'Liverpool Street', 'Highgate']
New station line ['Queens Park', 'Swiss Cottage', 'Highgate', 'Moorgate', 'Liverpool Street', 'Chigwell']
Total track length from Queens Park to Chigwell : 18.6686 miles
Execution time newLine: 0.0


Station list ['Abbey Road', 'Barbican', 'Bethnal Green', 'Cambridge Heath', 'Covent Garden', 'Dollis Hill', 'East Finchley', 'Finchley Road and Frognal', 'Great Portland